In [4]:
! pip install kaggle

Uploaing Kaggle.json file

In [3]:
#Configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing X Sentiment dataset

In [5]:
#API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140  #use kaggle to look for sentiment140 dataset

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 90% 73.0M/80.9M [00:01<00:00, 81.1MB/s]
100% 80.9M/80.9M [00:01<00:00, 73.2MB/s]


In [6]:
#Extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, "r") as zip:
  zip.extractall()
  print("The dataset is extracted.")

The dataset is extracted.


Importing the Dependencies

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Printing stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [9]:
#loading the data from csv file to pandas dataframe
X_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1")

In [10]:
#checking the numbers of row and column
X_data.shape

(1599999, 6)

In [11]:
#printing the first 5 rows of the dataframe
X_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


So, as we can see in the upper code its not reading the actual coloums but converting dataframe as an coloum, we need to name coloum and run it again.

In [12]:
#nameing the columns and running again

colum_name = ['target','id','date','flag','user','text']
X_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=colum_name, encoding = "ISO-8859-1")

In [13]:
#checking the numbers of row and column
X_data.shape

(1600000, 6)

In [14]:
#printing the first 5 rows of the dataframe
X_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
#counting the numbers of missing value in the dataset
X_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

Checking the distribution of the target column

In [17]:
X_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Converting the target "4" to "1"

In [18]:
X_data.replace({'target':{4:1}}, inplace=True)

In [19]:
#Checking the distribution of the target column
X_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet

1 --> Positive Tweet


**Stemming**

Stemming is the process of reducing the words to its roots word


example: actor,actress,acting = act

In [20]:
port_stem = PorterStemmer()

In [21]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content



In [23]:
X_data['stemmed_content'] = X_data['text'].apply(stemming)  #50 minutes to complete

In [24]:
X_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [26]:
print(X_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [27]:
print(X_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [28]:
#seprating the data and the label
X = X_data['stemmed_content'].values
Y = X_data['target'].values

In [29]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [30]:
print(Y)

[0 0 0 ... 1 1 1]


Spliting the data into the training data and test data

In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 5)

In [45]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [46]:
print(X_train)

['morn hope everyon great day'
 'nadhiyamali invit radha best chai ur mommi make weird radha invit basanti n veeru'
 'sparkli pattyfarm thank rt' ...
 'catatoniccat keep hair check roof look like got fro wear hat'
 'work ruin life miss everyth work'
 'still work half compani ballgam other dinner']


In [47]:
print(X_test)


['emili like pee deep end ahhaah'
 'mean see aaaaaaaaand way dailybooth meet best friend http tinyurl com mwsm w'
 'go ring shop ladi' ... 'nfrussel yeah road trip'
 'ptinsley damn suck load librari yet test music command ruin day'
 'mileycyru new song come along good hope hope well famili god bless chanc repli']


Conveting the textual data into Numerical data

In [48]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [49]:
print(X_train)

  (0, 93843)	0.3520625811633061
  (0, 153268)	0.4594366485420904
  (0, 124737)	0.5231197021850295
  (0, 169423)	0.4307460563004172
  (0, 274638)	0.45362265936406476
  (1, 429569)	0.3606951529447222
  (1, 33752)	0.3606951529447222
  (1, 438493)	0.17920320711015286
  (1, 249196)	0.11625131115897648
  (1, 272655)	0.1922328392689911
  (1, 426531)	0.1476075287094744
  (1, 66382)	0.2524858074543787
  (1, 39493)	0.1402323392193017
  (1, 331003)	0.5650798659991657
  (1, 181713)	0.37735485447514094
  (1, 283017)	0.30050337168173064
  (2, 346877)	0.4461921387303361
  (2, 400406)	0.21359272849862912
  (2, 310001)	0.6920301869860533
  (2, 377901)	0.5257232560652112
  (3, 397282)	0.34890525194721284
  (3, 102415)	0.420743879737433
  (3, 78450)	0.3348573915248513
  (3, 127112)	0.3120936733238297
  (3, 2485)	0.3822125370657282
  :	:
  (1279996, 438991)	0.31219774225859287
  (1279996, 445613)	0.21229083138944874
  (1279997, 344980)	0.36528700223303945
  (1279997, 63204)	0.49356198783242905
  (1279997,

In [52]:
print(X_test)

  (0, 311854)	0.4171144373494487
  (0, 234671)	0.18241073412253994
  (0, 120039)	0.27350243321462087
  (0, 118629)	0.39959613334366184
  (0, 96204)	0.3883553643040905
  (0, 6647)	0.6383113115794222
  (1, 436990)	0.21823309985776707
  (1, 409728)	0.272543871815237
  (1, 358217)	0.18851425508300926
  (1, 259198)	0.25961146603053226
  (1, 258502)	0.2499920169561539
  (1, 171195)	0.1742967177513577
  (1, 138886)	0.21710676644564528
  (1, 88887)	0.4596247711759963
  (1, 78763)	0.1870233790393096
  (1, 39493)	0.2358289629722106
  (1, 98)	0.5773211737315743
  (2, 365683)	0.4956902588423947
  (2, 341209)	0.6190504487881319
  (2, 225636)	0.5431230906491545
  (2, 149798)	0.2758351274723409
  (3, 324595)	0.20184426143574094
  (3, 319447)	0.26381790544732014
  (3, 283716)	0.26568852073370863
  (3, 261875)	0.34469144514205413
  :	:
  (319997, 342468)	0.6622239623228642
  (319998, 454554)	0.221774674250392
  (319998, 399548)	0.26065775606923364
  (319998, 387191)	0.21599396634117232
  (319998, 34740

Training the Machine Learning Model

Logistic Regression

In [53]:
model  = LogisticRegression(max_iter = 1000)

model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [55]:
#Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)


In [56]:
print("Accuracy score on the training data :", training_data_accuracy)

Accuracy score on the training data : 0.810353125


In [63]:
#Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [64]:
print("Accuracy score on the test data :", test_data_accuracy)


Accuracy score on the test data : 0.777621875


Model Accuracy = 77.8%

**Saving Trained Model**

In [66]:
import pickle

In [67]:
filename = "train_model.sav"
pickle.dump(model, open(filename, "wb"))


Uing the Saved Model for futur prediction

In [ ]:
#loading the saved model
load_model = pickle.load(open("/content/train_model.sav","rb"))

In [71]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("It is a Negative Tweet")
else:
  print("It is a Positive Tweet")

1
[1]
It is a Positive Tweet


In [73]:
X_new = X_test[83]
print(Y_test[83])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("It is a Negative Tweet")
else:
  print("It is a Positive Tweet")

0
[0]
It is a Negative Tweet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')